# Aplicação das transformações no CV e Test set

In [8]:
"""
Script para aplicar a pipeline de pré-processamento nos conjuntos de treino, validação e teste,
garantindo que as mesmas transformações sejam aplicadas em todos.
"""

import os
import sys
import pandas as pd
import numpy as np
import joblib
import warnings

# Adiciona diretório pai ao path para encontrar os módulos
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

warnings.filterwarnings('ignore')

# Imports dos módulos de pré-processamento
from src.preprocessing.email_processing import normalize_emails_in_dataframe
from src.preprocessing.data_cleaning import (
    consolidate_quality_columns,
    handle_missing_values,
    handle_outliers,
    normalize_values,
    convert_data_types
)
from src.preprocessing.feature_engineering import feature_engineering
from src.preprocessing.text_processing import text_feature_engineering

def apply_preprocessing_pipeline(df, params=None, fit=False):
    """
    Aplica a pipeline completa de pré-processamento.
    
    Args:
        df: DataFrame a ser processado
        params: Parâmetros para transformações (None para começar do zero)
        fit: Se True, ajusta as transformações, se False, apenas aplica
        
    Returns:
        DataFrame processado e parâmetros atualizados
    """
    # Inicializar parâmetros se não fornecidos
    if params is None:
        params = {}
    
    print(f"Iniciando pipeline de pré-processamento para DataFrame: {df.shape}")
    
    # 1. Normalizar emails
    print("1. Normalizando emails...")
    df = normalize_emails_in_dataframe(df, email_col='email')
    
    # 2. Consolidar colunas de qualidade
    print("2. Consolidando colunas de qualidade...")
    quality_params = params.get('quality_columns', {})
    df, quality_params = consolidate_quality_columns(df, fit=fit, params=quality_params)
    
    # 3. Tratamento de valores ausentes
    print("3. Tratando valores ausentes...")
    missing_params = params.get('missing_values', {})
    df, missing_params = handle_missing_values(df, fit=fit, params=missing_params)
    
    # 4. Tratamento de outliers
    print("4. Tratando outliers...")
    outlier_params = params.get('outliers', {})
    df, outlier_params = handle_outliers(df, fit=fit, params=outlier_params)
    
    # 5. Normalização de valores
    print("5. Normalizando valores numéricos...")
    norm_params = params.get('normalization', {})
    df, norm_params = normalize_values(df, fit=fit, params=norm_params)
    
    # 6. Converter tipos de dados
    print("6. Convertendo tipos de dados...")
    df, _ = convert_data_types(df, fit=fit)
    
    # 7. Feature engineering não-textual
    print("7. Aplicando feature engineering não-textual...")
    feature_params = params.get('feature_engineering', {})
    df, feature_params = feature_engineering(df, fit=fit, params=feature_params)
    
    # 8. Processamento de texto
    print("8. Processando features textuais...")
    text_params = params.get('text_processing', {})
    df, text_params = text_feature_engineering(df, fit=fit, params=text_params)
    
    # 9. Compilar parâmetros atualizados
    updated_params = {
        'quality_columns': quality_params,
        'missing_values': missing_params,
        'outliers': outlier_params,
        'normalization': norm_params,
        'feature_engineering': feature_params,
        'text_processing': text_params
    }
    
    print(f"Pipeline concluída! Dimensões finais: {df.shape}")
    return df, updated_params

def ensure_column_consistency(train_df, test_df):
    """
    Garante que o DataFrame de teste tenha as mesmas colunas que o de treinamento.
    
    Args:
        train_df: DataFrame de treinamento
        test_df: DataFrame de teste para alinhar
        
    Returns:
        DataFrame de teste com colunas alinhadas
    """
    print("Alinhando colunas entre conjuntos de dados...")
    
    # Colunas presentes no treino, mas ausentes no teste
    missing_cols = set(train_df.columns) - set(test_df.columns)
    
    # Adicionar colunas faltantes com valores padrão
    for col in missing_cols:
        if col in train_df.select_dtypes(include=['number']).columns:
            test_df[col] = 0
        else:
            test_df[col] = None
        print(f"  Adicionada coluna ausente: {col}")
    
    # Remover colunas extras no teste não presentes no treino
    extra_cols = set(test_df.columns) - set(train_df.columns)
    if extra_cols:
        test_df = test_df.drop(columns=list(extra_cols))
        print(f"  Removidas colunas extras: {', '.join(list(extra_cols)[:5])}" + 
              (f" e mais {len(extra_cols)-5} outras" if len(extra_cols) > 5 else ""))
    
    # Garantir a mesma ordem de colunas
    test_df = test_df[train_df.columns]
    
    print(f"Alinhamento concluído: {len(missing_cols)} colunas adicionadas, {len(extra_cols)} removidas")
    return test_df

# Função principal - adaptada para notebook
def process_datasets():
    """
    Função principal que processa todos os conjuntos na ordem correta.
    """
    # 1. Definir caminhos dos datasets
    # Considerando que estamos em smart_ads/notebooks e precisamos acessar smart_ads/data/split2
    base_dir = os.path.dirname(os.getcwd())  # Sobe um nível para smart_ads
    base_data_dir = os.path.join(base_dir, "data", "split2")
    
    train_path = os.path.join(base_data_dir, "train0.csv")
    cv_path = os.path.join(base_data_dir, "validation.csv")
    test_path = os.path.join(base_data_dir, "test.csv")
    
    # Garantir que o diretório de saída existe
    output_dir = os.path.join(os.getcwd(), "datasets", "split2")
    os.makedirs(output_dir, exist_ok=True)
    
    # Verificar se os arquivos existem
    print(f"Verificando existência dos arquivos de entrada:")
    print(f"  Train path: {train_path} - Existe: {os.path.exists(train_path)}")
    print(f"  CV path: {cv_path} - Existe: {os.path.exists(cv_path)}")
    print(f"  Test path: {test_path} - Existe: {os.path.exists(test_path)}")
    
    # 2. Carregar os datasets
    print(f"Carregando datasets de {base_data_dir}...")
    train_df = pd.read_csv(train_path)
    cv_df = pd.read_csv(cv_path)
    test_df = pd.read_csv(test_path)
    
    print(f"Datasets carregados: treino {train_df.shape}, validação {cv_df.shape}, teste {test_df.shape}")
    
    # 3. Processar o conjunto de treinamento com fit=True para aprender parâmetros
    print("\n--- Processando conjunto de treinamento ---")
    train_processed, params = apply_preprocessing_pipeline(train_df, fit=True)
    
    # 4. Salvar parâmetros aprendidos
    params_dir = os.path.join(current_dir, "models", "preprocessing_params")
    os.makedirs(params_dir, exist_ok=True)
    
    joblib.dump(params, os.path.join(params_dir, "all_preprocessing_params.joblib"))
    print(f"Parâmetros de pré-processamento salvos em {params_dir}/all_preprocessing_params.joblib")
    
    # 5. Salvar conjunto de treino processado
    train_processed.to_csv(os.path.join(output_dir, "train.csv"), index=False)
    print(f"Dataset de treino processado e salvo em {output_dir}/train.csv")
    
    # 6. Processar o conjunto de validação com fit=False para aplicar parâmetros aprendidos
    print("\n--- Processando conjunto de validação ---")
    cv_processed, _ = apply_preprocessing_pipeline(cv_df, params=params, fit=False)
    
    # 7. Garantir consistência de colunas com o treino
    cv_processed = ensure_column_consistency(train_processed, cv_processed)
    
    # 8. Salvar conjunto de validação processado
    cv_processed.to_csv(os.path.join(output_dir, "validation.csv"), index=False)
    print(f"Dataset de validação processado e salvo em {output_dir}/validation.csv")
    
    # 9. Processar o conjunto de teste com fit=False para aplicar parâmetros aprendidos
    print("\n--- Processando conjunto de teste ---")
    test_processed, _ = apply_preprocessing_pipeline(test_df, params=params, fit=False)
    
    # 10. Garantir consistência de colunas com o treino
    test_processed = ensure_column_consistency(train_processed, test_processed)
    
    # 11. Salvar conjunto de teste processado
    test_processed.to_csv(os.path.join(output_dir, "test.csv"), index=False)
    print(f"Dataset de teste processado e salvo em {output_dir}/test.csv")
    
    print("\nPré-processamento dos conjuntos concluído com sucesso!")
    print(f"Os datasets processados foram salvos em {output_dir}/")
    
    return {
        'train': train_processed,
        'cv': cv_processed,
        'test': test_processed,
        'params': params
    }

# Execute a função principal
results = process_datasets()

Verificando existência dos arquivos de entrada:
  Train path: /home/jupyter/smart_ads/data/split2/train0.csv - Existe: True
  CV path: /home/jupyter/smart_ads/data/split2/validation.csv - Existe: True
  Test path: /home/jupyter/smart_ads/data/split2/test.csv - Existe: True
Carregando datasets de /home/jupyter/smart_ads/data/split2...
Datasets carregados: treino (97174, 48), validação (20823, 48), teste (20823, 48)

--- Processando conjunto de treinamento ---
Iniciando pipeline de pré-processamento para DataFrame: (97174, 48)
1. Normalizando emails...
2. Consolidando colunas de qualidade...
3. Tratando valores ausentes...
4. Tratando outliers...
5. Normalizando valores numéricos...
6. Convertendo tipos de dados...
7. Aplicando feature engineering não-textual...
8. Processando features textuais...
Pipeline concluída! Dimensões finais: (97174, 404)
Parâmetros de pré-processamento salvos em /home/jupyter/smart_ads/notebooks/models/preprocessing_params/all_preprocessing_params.joblib
Datase

# Feature importance and selection no cv set e training set

In [11]:
# Feature Importance e Feature Selection Pipeline
# Adaptado do script feature_importance_pipeline.py para uso em notebook

import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Imports dos módulos de avaliação
from src.evaluation import feature_importance as fi
from src.evaluation import feature_selector as fs

# Criar pasta para resultados
output_dir = 'reports/eda_results/feature_importance_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
    print(f"Pasta '{output_dir}' criada para salvar resultados")
else:
    print(f"Pasta '{output_dir}' já existe")

# 1 - Carregar dataset
print("Carregando dataset...")
data_dir = '../data/split3'
train_path = os.path.join(data_dir, "train.csv")
cv_path = os.path.join(data_dir, "validation.csv") 
test_path = os.path.join(data_dir, "test.csv")

try:
    # Tentar carregar os arquivos individualmente
    df_list = []
    if os.path.exists(train_path):
        train_df = pd.read_csv(train_path)
        df_list.append(train_df)
        print(f"Dataset de treino carregado: {train_df.shape}")
    
    if os.path.exists(cv_path):
        cv_df = pd.read_csv(cv_path)
        df_list.append(cv_df)
        print(f"Dataset de validação carregado: {cv_df.shape}")
    
    if os.path.exists(test_path):
        test_df = pd.read_csv(test_path)
        df_list.append(test_df)
        print(f"Dataset de teste carregado: {test_df.shape}")
    
    # Se pelo menos um arquivo foi carregado, combine-os
    if df_list:
        df = pd.concat(df_list, ignore_index=True)
        print(f"Dataset combinado: {df.shape[0]} linhas, {df.shape[1]} colunas")
    else:
        raise FileNotFoundError("Nenhum arquivo encontrado em data/split3")
except Exception as e:
    print(f"Erro ao carregar datasets: {e}")
    # Tentar carregar arquivos da pasta notebooks/datasets
    try:
        notebook_train_path = "datasets/split/train.csv"
        if os.path.exists(notebook_train_path):
            df = pd.read_csv(notebook_train_path)
            print(f"Dataset alternativo carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")
        else:
            # Última tentativa - arquivos originais
            alt_paths = ["smart_ads_all_features.csv", "smart_ads_cleaned.csv"]
            df_loaded = False
            for path in alt_paths:
                try:
                    df = pd.read_csv(path)
                    df_loaded = True
                    print(f"Arquivo alternativo {path} carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")
                    break
                except:
                    print(f"Arquivo alternativo {path} não encontrado.")
            
            if not df_loaded:
                raise FileNotFoundError("Não foi possível encontrar nenhum dataset para análise")
    except Exception as e2:
        print(f"Erro final ao tentar carregar qualquer dataset: {e2}")
        raise

# 2 - Identificar colunas de lançamento e target
print("\nIdentificando colunas importantes...")
launch_col = fi.identify_launch_column(df)
target_col = fi.identify_target_column(df)

# 3 - Selecionar features numéricas para análise
numeric_cols = fi.select_numeric_features(df, target_col)

# 4 - Identificar colunas derivadas de texto
text_derived_cols = fi.identify_text_derived_columns(numeric_cols)

# 5 - Sanitizar nomes de colunas
rename_dict = fi.sanitize_column_names(numeric_cols)

# Aplicar renomeação se necessário
if rename_dict:
    print(f"Renomeando {len(rename_dict)} colunas para evitar erros com caracteres especiais")
    df = df.rename(columns=rename_dict)
    
    # Atualizar listas
    numeric_cols = [rename_dict.get(col, col) for col in numeric_cols]
    text_derived_cols = [rename_dict.get(col, col) for col in text_derived_cols]

# 6 - Preparar dados para modelagem
X = df[numeric_cols].fillna(0)
y = df[target_col]

print(f"Usando {len(numeric_cols)} features numéricas para análise")
print(f"Distribuição do target: {y.value_counts(normalize=True) * 100}")

# 7 - Análise de multicolinearidade
high_corr_pairs = fi.analyze_multicollinearity(X)

# 8 - Análise específica: comparação de encodings de país
fi.compare_country_encodings(X, y)

# 9 - Separar dados para treinamento e validação
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Treinamento: {X_train.shape[0]} amostras, Validação: {X_val.shape[0]} amostras")
print(f"Proporção da classe positiva no treino: {y_train.mean()*100:.2f}%")
print(f"Proporção da classe positiva na validação: {y_val.mean()*100:.2f}%")

# 10 - Análise de importância com múltiplos modelos
print("\n--- Iniciando análise de importância de features ---")

# 10.1 - RandomForest
rf_importance, rf_metrics = fi.analyze_rf_importance(X, y, numeric_cols)

# 10.2 - LightGBM
lgb_importance, lgb_metrics = fi.analyze_lgb_importance(X, y, numeric_cols)

# 10.3 - XGBoost
xgb_importance, xgb_metrics = fi.analyze_xgb_importance(X, y, numeric_cols)

# 11 - Combinar resultados de importância
final_importance = fi.combine_importance_results(rf_importance, lgb_importance, xgb_importance)

# 12 - Salvar resultados combinados
final_importance.to_csv(os.path.join(output_dir, 'feature_importance_combined.csv'), index=False)
print(f"\nImportância das features salva em {os.path.join(output_dir, 'feature_importance_combined.csv')}")

# 13 - Análise de robustez entre lançamentos
launch_importance, unstable_features, launch_vs_global, consistent_features = fi.analyze_launch_robustness(
    df, X, y, numeric_cols, launch_col, rename_dict, final_importance
)

# Salvar análise de robustez entre lançamentos se disponível
if launch_vs_global is not None:
    launch_vs_global.to_csv(os.path.join(output_dir, 'feature_robustness_analysis.csv'), index=False)
    print(f"\nAnálise de robustez entre lançamentos salva em {os.path.join(output_dir, 'feature_robustness_analysis.csv')}")

# 14 - Identificar features potencialmente irrelevantes
potentially_irrelevant, irrelevant_by_importance, irrelevant_by_variance, irrelevant_by_correlation = fs.identify_irrelevant_features(
    final_importance, high_corr_pairs
)

# 15 - Análise de features textuais
text_importance = fs.analyze_text_features(final_importance, text_derived_cols)

# Salvar análise de features textuais se disponível
if text_importance is not None:
    text_importance.to_csv(os.path.join(output_dir, 'text_features_importance.csv'), index=False)
    print(f"Análise de features textuais salva em {os.path.join(output_dir, 'text_features_importance.csv')}")

# 16 - Selecionar features finais e criar recomendações
original_relevant_features, features_to_remove_corr, unrecommended_features = fs.select_final_features(
    final_importance, high_corr_pairs, numeric_cols, rename_dict
)

# 17 - Documentar seleções de features
fs.document_feature_selections(
    original_relevant_features, unrecommended_features, 
    final_importance, high_corr_pairs, rename_dict, output_dir
)

# 18 - Criar datasets com features selecionadas
# Determinar caminhos de output para datasets processados
output_data_dir = 'data/split3'
if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir, exist_ok=True)

# Conjunto de treino (se existir)
if os.path.exists(train_path):
    train_output_path = os.path.join(output_data_dir, "train_selected.csv")
    fs.create_selected_dataset(
        original_relevant_features, target_col, train_path, train_output_path
    )
    print(f"Dataset de treino com features selecionadas salvo em '{train_output_path}'")

# Conjunto de validação (se existir)
if os.path.exists(cv_path):
    cv_output_path = os.path.join(output_data_dir, "validation_selected.csv")
    fs.create_selected_dataset(
        original_relevant_features, target_col, cv_path, cv_output_path
    )
    print(f"Dataset de validação com features selecionadas salvo em '{cv_output_path}'")

# Conjunto de teste (se existir)
if os.path.exists(test_path):
    test_output_path = os.path.join(output_data_dir, "test_selected.csv")
    fs.create_selected_dataset(
        original_relevant_features, target_col, test_path, test_output_path
    )
    print(f"Dataset de teste com features selecionadas salvo em '{test_output_path}'")

# 19 - Resumir categorias de features
fs.summarize_feature_categories(numeric_cols, final_importance, text_derived_cols)

print("\nAnálise de importância de features concluída com sucesso!")
print(f"Todos os resultados foram salvos na pasta '{output_dir}'")
print(f"Datasets com features selecionadas salvos em '{output_data_dir}'")

# Variáveis importantes para uso posterior no notebook
results = {
    'df': df,
    'numeric_cols': numeric_cols,
    'text_derived_cols': text_derived_cols,
    'final_importance': final_importance,
    'selected_features': original_relevant_features
}

Pasta 'reports/eda_results/feature_importance_results' já existe
Carregando dataset...
Dataset de treino carregado: (97174, 404)
Dataset de validação carregado: (20823, 404)
Dataset de teste carregado: (20823, 404)
Dataset combinado: 138820 linhas, 404 colunas

Identificando colunas importantes...
Coluna de lançamento encontrada: 'lançamento'
Número de lançamentos: 6
Lançamentos identificados: ['L16', 'L17', 'L18', 'L19', 'L20', 'L21']
Distribuição de lançamentos:
lançamento
L21    22.577438
L16    20.629592
L17    16.944965
L20    14.325746
L19    12.839648
L18    12.682611
Name: proportion, dtype: float64
Removendo 2 colunas com variância zero
Features derivadas de texto identificadas: 270
Exemplos de features textuais:
  - name_length
  - name_word_count
  - Cuando hables inglés con fluidez, ¿qué cambiará en tu vida? ¿Qué oportunidades se abrirán para ti?_length
  - Cuando hables inglés con fluidez, ¿qué cambiará en tu vida? ¿Qué oportunidades se abrirán para ti?_word_count
  - Cuan